In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

In [2]:
# Load and preprocess data
data = pd.read_csv("NFLX_DATA2023.csv")
data = data[['Open', 'High', 'Low', 'Volume', 'Close']]
#min-max normalisation
normalized_df = (data - data.min()) / (data.max() - data.min())

X = normalized_df.drop('Close', axis=1).values
y = normalized_df['Close'].values.reshape(-1, 1)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
Y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
Y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [5]:
# Model definition
class LinearRegression(nn.Module):
    def __init__(self, input_features):
        super(LinearRegression, self).__init__()
        self.layer = nn.Linear(input_features, 1)

    def forward(self, X):
        return self.layer(X)

input_features = X_train.shape[1]
model = LinearRegression(input_features)

In [6]:
# Loss function and optimizer
loss_func = nn.MSELoss()
optimiser = torch.optim.SGD(model.parameters(), lr=0.01)

In [7]:
# Training loop
epochs = 1000
for epoch in range(epochs):
    model.train()
    pred = model(X_train_tensor)
    loss = loss_func(pred, Y_train_tensor)

    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.0143
Epoch [200/1000], Loss: 0.0047
Epoch [300/1000], Loss: 0.0026
Epoch [400/1000], Loss: 0.0016
Epoch [500/1000], Loss: 0.0011
Epoch [600/1000], Loss: 0.0008
Epoch [700/1000], Loss: 0.0006
Epoch [800/1000], Loss: 0.0006
Epoch [900/1000], Loss: 0.0005
Epoch [1000/1000], Loss: 0.0005


In [8]:
# Evaluation
model.eval()
with torch.inference_mode():
    test_predictions = model(X_test_tensor).squeeze()
    test_loss = loss_func(test_predictions, Y_test_tensor.squeeze())

    # Convert tensors to NumPy arrays for test metrics
    test_predictions_np = test_predictions.numpy()
    Y_test_np = Y_test_tensor.squeeze().numpy()

    # Calculate MAE and R-squared
    test_mae = mean_absolute_error(Y_test_np, test_predictions_np)
    test_r2 = r2_score(Y_test_np, test_predictions_np)

    print(f'Test Loss (MSE): {test_loss.item():.4f}')
    print(f'Test MAE: {test_mae:.4f}')
    print(f'Test R-squared: {test_r2:.4f}')



Test Loss (MSE): 0.0005
Test MAE: 0.0149
Test R-squared: 0.9915
